# Solution Architecture

```
                ┌─────────────────┐
                │   FastAPI App   │
                │  (api/main.py)  │
                └─────────────────┘
                         │
          ┌──────────────┼──────────────┐
          ▼              ▼              ▼
  ┌─────────────┐ ┌─────────────┐ ┌─────────────┐
  │ Prediction  │ │   Model     │ │    Data     │
  │  Service    │ │ Management  │ │  Validation │
  │(services.py)│ │   Layer     │ │ (models.py) │
  └─────────────┘ └─────────────┘ └─────────────┘
          │              │              │
          └──────────────┼──────────────┘
                         ▼
                ┌─────────────────┐
                │   Data Layer    │
                │ (Files & CSVs)  │
                └─────────────────┘
```

**File Structure & Responsabilities**

```
ml-phdata/
├── api/                            # Main application package
│   ├── __init__.py                 # Package marker
│   ├── main.py                     # FastAPI app & endpoints
│   ├── models.py                   # Pydantic data models
│   ├── services.py                 # Business logic layer
│   └── config.py                   # Configuration management
├── data/                           # Training data storage
│   ├── kc_house_data.csv           # House sales data
│   └── zipcode_demographics.csv    # Demographic data
├── model/                          # ML model artifacts
│   ├── model.pkl                   # Trained model
│   ├── model_backup.pkl            # Backup for rollback
│   └── model_features.json         # Feature definitions
├── create_model.py                 # Initial model training script
├── run.py                          # Application entry point
├── requirements.txt                # Python dependencies
├── Dockerfile                      # Container configuration
└── README.md                       # Project documentation
```

**1. API Layer (api/main.py)**

```
  ┌─────────────────────────────────────────┐
  │              REST Endpoints             │
  ├─────────────────────────────────────────┤
  │ GET  /features/required                 │
  │ POST /predict                           │
  │ POST /predict-batch                     │
  │ POST /reload-model                      │
  │ POST /retrain-model                     │
  │ POST /rollback-model                    │
  └─────────────────────────────────────────┘

  Responsibilities:
  - HTTP request/response handling
  - Input validation via Pydantic models
  - Error handling and status codes
  - API documentation (FastAPI auto-generates OpenAPI/Swagger)
  - Logging and monitoring hooks
```

**2. Business Logic Layer (api/services.py)**

```
  ┌─────────────────────────────────────────┐
  │            Service Classes              │
  ├─────────────────────────────────────────┤
  │ TrainService                            │
  │  ├─ Data loading & preprocessing        │
  │  ├─ Model training & evaluation         │
  │  └─ Model persistence                   │
  │                                         │
  │ PredictionService                       │
  │  ├─ Model loading & caching             │
  │  ├─ Feature preprocessing               │
  │  ├─ Prediction inference                │
  │  ├─ Demographic data joining            │
  │  └─ Model lifecycle management          │
  └─────────────────────────────────────────┘

  Key Design Patterns:
  - Service Layer Pattern: Encapsulates business logic
  - Singleton Pattern: Global prediction service instance
  - Template Method: Common prediction workflow with variations
```

**3. Data Model Layer (api/models.py)**

```
  ┌─────────────────────────────────────────┐
  │           Pydantic Models               │
  ├─────────────────────────────────────────┤
  │ Request Models:                         │
  │  ├─ PredictionRequest (8 features)      │
  │  ├─ MinimalFeatures (planned)           │
  │  └─ CompleteFeatures (planned)          │
  │                                         │
  │ Response Models:                        │
  │  ├─ PredictionResponse                  │
  │  ├─ BatchPredictionResponse             │
  │  └─ RequiredFeaturesResponse            │
  │                                         │
  │ Metadata Models:                        │
  │  └─ PredictionMetadata                  │
  └─────────────────────────────────────────┘

  Benefits:
  - Type Safety: Runtime validation and IDE support
  - API Documentation: Auto-generated schemas
  - Serialization: JSON conversion handling
  - Data Integrity: Field validation and constraints
```

**4. Configuration Layer (api/config.py)**

```
  ┌─────────────────────────────────────────┐
  │        Configuration Management         │
  ├─────────────────────────────────────────┤
  │ File Paths:                             │
  │  ├─ MODEL_PATH                          │
  │  ├─ FEATURES_PATH                       │
  │  ├─ ZIPCODE_DATA_PATH                   │
  │  └─ SALES_DATA_PATH                     │
  │                                         │
  │ Application Settings:                   │
  │  ├─ SALES_COLUMN_SELECTION              │
  │  ├─ TARGET variable                     │
  │  ├─ API_TITLE & VERSION                 │
  │  └─ Logging configuration               │
  └─────────────────────────────────────────┘
```

**5. Data Layer**

```
  ┌─────────────────────────────────────────┐
  │              Data Storage               │
  ├─────────────────────────────────────────┤
  │ ML Artifacts:                           │
  │  ├─ model/model.pkl (trained model)     │
  │  ├─ model/model_backup.pkl (backup)     │
  │  └─ model/model_features.json           │
  │                                         │
  │ Training Data:                          │
  │  ├─ data/kc_house_data.csv              │
  │  └─ data/zipcode_demographics.csv       │
  └─────────────────────────────────────────┘
```

**6. Docker Usage**

- Environment Consistency
- Simplified Deployment
- Easy Testing and Debugging

# Current Limitations

**Single Point of Failure:**  
  - The API runs as a single FastAPI instance with a global prediction_service  
  - Model is loaded into memory on startup  
  - Any restart means service downtime

**Memory & Resource Constraints:**
  - Each instance loads the full model into memory
  - Synchronous processing limits concurrent requests
  - File-based model storage creates I/O bottlenecks

**Model Management Issues:**
  - Model updates require service restart
  - No versioning or rollback capabilities during high traffic
  - Backup/restore operations are risky in production

**Security**
  - No authentication/authorization
  - No rate limiting
  - No metrics/monitoring
  - No database connections
  - No secrets management

# Scalable Architecture Solutions

- Docker containers for consistent deployment
- Kubernetes/Docker Swarm for orchestration
- Models stored in object storage (S3, GCS) with versioning
- Prediction caching
- Replace CSV with proper database

# Zero-Downtime Scaling Solution

1. Keep current version (blue) running
2. Deploy new version (green) in parallel
3. Gradually shift traffic using load balancer
4. Monitor metrics and rollback if needed
5. Terminate old version once stable